<a href="https://colab.research.google.com/github/algharak/semi-demand-fcst/blob/master/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
!pip install mxnet
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from itertools import islice
from pathlib import Path


     |████████████████████████████████| 68.7MB 52kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
mx.random.seed(0)
np.random.seed(0)

In [3]:
!pip install gluonts

     |████████████████████████████████| 419kB 2.7MB/s 
     |████████████████████████████████| 194kB 8.1MB/s 
     |████████████████████████████████| 7.4MB 10.2MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68025 sha256=afef06d779568fd30c4621299f82e865ef474f455d167301c8e00a3c3503e6b4
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


In [4]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas
print(f"Available datasets: {list(dataset_recipes.keys())}")

Available datasets: ['constant', 'exchange_rate', 'solar-energy', 'electricity', 'traffic', 'exchange_rate_nips', 'electricity_nips', 'traffic_nips', 'solar_nips', 'wiki-rolling_nips', 'taxi_30min', 'm4_hourly', 'm4_daily', 'm4_weekly', 'm4_monthly', 'm4_quarterly', 'm4_yearly', 'm5']


In [5]:
dataset = get_dataset("m4_monthly", regenerate=True)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


In [32]:
train_entry = next(iter(dataset.train))
#train_entry = next(iter(dataset.train))
aa = train_entry
print (aa['item_id'])
print(train_entry.keys())


0
dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])


In [40]:
train_entry = next(iter(dataset.train))
test_entry = next(iter(dataset.test))
print(train_entry['target'].shape)

print(test_entry['target'].shape)
train_entry = next(iter(dataset.train))
test_entry = next(iter(dataset.test))
print(train_entry['target'].shape)

print(test_entry['target'].shape)


(469,)
(487,)
(469,)
(487,)


In [11]:
print (type(dataset.metadata))
print (dataset.metadata)


<class 'gluonts.dataset.common.MetaData'>
freq='M' target=None feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=18


In [ ]:
me=iter(dataset.train)
j = 1
for i in me:
  mef = to_pandas(i)
  print (j)
  print (mef.shape)
  j +=1
print ('i am donr')



In [ ]:

fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))

train_series.plot(ax=ax[0])
ax[0].grid(which="both")
ax[0].legend(["train series"], loc="upper left")

test_series.plot(ax=ax[1])
ax[1].axvline(train_series.index[-1], color='r') # end of train dataset
ax[1].grid(which="both")
ax[1].legend(["test series", "end of train series"], loc="upper left")

plt.show()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [7]:
import io
import pandas as pd

df2 = pd.read_csv(io.BytesIO(uploaded['Historical Product Demand.csv']))
df2.head

NameError: ignored

In [4]:
df2.shape

NameError: ignored

In [86]:
colnames = df2.columns.to_list()
print(type(colnames))
for i in colnames:
  print (i)

<class 'list'>
Product_Code
Warehouse
Product_Category
Date
Order_Demand


In [90]:
#print (pd.unique(df2['Product_Category']))
prod_categ = list(pd.unique(df2['Product_Category']))
print(type(prod_categ))

<class 'list'>


In [28]:
start_date = df2['Date'].min()
end_date = df2['Date'].max()
print (start_date,end_date)

2011-01-08 00:00:00 2017-01-09 00:00:00


In [29]:
idx = pd.date_range(start=start_date, end=end_date)

In [40]:
dtdelta = end_date-start_date
nrows = dtdelta.days
print (type(nrows))
ncols = len (colnames)
print(nrows,ncols)

<class 'int'>
2193 33


In [3]:
dfnew = pd.DataFrame(np.zeros((nrows+1,ncols),dtype=int),index=idx, columns=colnames)
dfnew.head

NameError: ignored

In [2]:
import datetime
cat_idx = colnames.index('Product_Category')
dt_idx = colnames.index('Date')
value_idx = colnames.index('Order_Demand')
print(col_idx)
df2.info()
df2['Date'] = df2['Date'].astype(str)
nr = df2.shape[0]
print(nr)
inds = pd.isnull(df2).any(1)
print('inds',inds)
for n in range(10):
  act_date = datetime.datetime.strptime(df2.iloc[n,dt_idx], '%Y-%m-%d').date()
  act_cat = df2.iloc[n,cat_idx]
  act_value = int(df2.iloc[n,value_idx])
  print (n)
  dfnew.loc[act_date,act_cat] += act_value
  #print (dfnew.loc[act_date,act_cat])
  #print(act_value)

NameError: ignored